In [3]:
import numpy as np
import pandas as pd

In [4]:
import math
import random
import re
from collections import Counter

### Constructing typical English incidence matrix
• Entries are probabilities with which two letters typically follow each other  
• Bigram frequencies used are from http://practicalcryptography.com/media/cryptanalysis/files/english_bigrams_1.txt

In [5]:
bigram_frequencies = pd.read_csv('english_bigrams.txt', sep=' ', header=None)
bigram_frequencies.columns = ['bigram', 'frequency']
bigram_frequencies['first'] = bigram_frequencies.bigram.str[0]
bigram_frequencies['second'] = bigram_frequencies.bigram.str[1]
bigram_frequencies['probability'] = bigram_frequencies.frequency/bigram_frequencies.frequency.sum()
bigram_frequencies.head()

,bigram,frequency,first,second,probability
0,TH,116997844,T,H,0.027057
1,HE,100689263,H,E,0.023285
2,IN,87674002,I,N,0.020276
3,ER,77134382,E,R,0.017838
4,AN,69775179,A,N,0.016136


In [6]:
bigram_frequencies_sorted = bigram_frequencies.sort_values('bigram') 
bigram_frequencies_sorted_freq = bigram_frequencies_sorted[['first', 'second', 'frequency']]
bigram_frequencies_sorted_prob = bigram_frequencies_sorted[['first', 'second', 'probability']]
bigram_frequencies_sorted_freq.head()

,first,second,frequency
323,A,A,1721143
141,A,B,8775582
71,A,C,17904683
89,A,D,14877234
383,A,E,815963


In [7]:
# probabilities 
typical_incidence_matrix_prob = bigram_frequencies_sorted_prob.set_index(['first', 'second']).unstack(0).T
typical_incidence_matrix_prob = typical_incidence_matrix_prob.iloc[1:] # del first row
typical_incidence_matrix_prob.drop(typical_incidence_matrix_prob.columns[0], axis=1, inplace=True) # del first col
#print(typical_incidence_matrix_prob.shape) # 26-by-26
#typical_incidence_matrix_prob.head()

In [8]:
# frequencies
typical_incidence_matrix_freq = bigram_frequencies_sorted_freq.set_index(['first', 'second']).unstack(0).T
typical_incidence_matrix_freq = typical_incidence_matrix_freq.iloc[1:] # del first row
typical_incidence_matrix_freq.drop(typical_incidence_matrix_freq.columns[0], axis=1, inplace=True) # del first col
typical_incidence_matrix_freq.head()

second                    A          B           C           D           E  \
          first                                                              
frequency A       1721143.0  8775582.0  17904683.0  14877234.0    815963.0   
          B       8867461.0   689158.0    320380.0    141752.0  19468489.0   
          C      19930754.0   298053.0   3026492.0    358435.0  19803619.0   
          D      17584055.0  6106719.0   3782481.0   4001275.0  27029835.0   
          E      43329810.0  9738798.0  25775798.0  46647960.0  18497942.0   

second                    F          G           H           I          J  \
          first                                                             
frequency A       5702567.0  8809266.0   2225270.0  13974919.0   870262.0   
          B         75352.0    40516.0    154489.0   4356462.0   282608.0   
          C        267630.0   181590.0  20132750.0   8446084.0    41526.0   
          D       4033878.0  2442139.0   4585765.0  21673998.0   799366.0   
          E      13252227.0  8286463.0   7559141.0  16026915.0  1256993.0   

second           ...          Q           R           S           T  \
          first  ...                                                  
frequency A      ...   315068.0  42353262.0  37773878.0  48274564.0   
          B      ...     5513.0   4621080.0   1409672.0    428276.0   
          C      ...   157546.0   5514347.0   1381608.0  11888752.0   
          D      ...   283314.0   5701879.0  10429887.0  15759673.0   
          E      ...  1461436.0  77134382.0  57070453.0  32872552.0   

second                   U           V           W          X           Y  \
          first                                                             
frequency A      4884168.0   8288885.0   3918960.0   660826.0  11523416.0   
          B      8113271.0    120081.0    140189.0     3021.0   5232074.0   
          C      4604045.0     94224.0    257253.0     5300.0   1145316.0   
          D      5861311.0   1238565.0   4906814.0    27413.0   2218040.0   
          E      3674130.0  10574011.0  14776406.0  5649363.0   7528342.0   

second                  Z  
          first            
frequency A      768359.0  
          B        8132.0  
          C       57914.0  
          D       98038.0  
          E      465466.0  

[5 rows x 26 columns]

In [9]:
letters = list(map(chr, range(65, 91))) # uppercase alphabet
typical_incidence_matrix_freq_clean = pd.DataFrame(typical_incidence_matrix_freq.values, index=letters, columns=letters)
typical_incidence_matrix_freq_arr = typical_incidence_matrix_freq_clean.values
#print(typical_incidence_matrix_freq_arr)

typical_incidence_matrix_freq_clean.head() # official English frequency bigram incidence matrix

,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
A,1721143.0,8775582.0,17904683.0,14877234.0,815963.0,5702567.0,8809266.0,2225270.0,13974919.0,870262.0,...,315068.0,42353262.0,37773878.0,48274564.0,4884168.0,8288885.0,3918960.0,660826.0,11523416.0,768359.0
B,8867461.0,689158.0,320380.0,141752.0,19468489.0,75352.0,40516.0,154489.0,4356462.0,282608.0,...,5513.0,4621080.0,1409672.0,428276.0,8113271.0,120081.0,140189.0,3021.0,5232074.0,8132.0
C,19930754.0,298053.0,3026492.0,358435.0,19803619.0,267630.0,181590.0,20132750.0,8446084.0,41526.0,...,157546.0,5514347.0,1381608.0,11888752.0,4604045.0,94224.0,257253.0,5300.0,1145316.0,57914.0
D,17584055.0,6106719.0,3782481.0,4001275.0,27029835.0,4033878.0,2442139.0,4585765.0,21673998.0,799366.0,...,283314.0,5701879.0,10429887.0,15759673.0,5861311.0,1238565.0,4906814.0,27413.0,2218040.0,98038.0
E,43329810.0,9738798.0,25775798.0,46647960.0,18497942.0,13252227.0,8286463.0,7559141.0,16026915.0,1256993.0,...,1461436.0,77134382.0,57070453.0,32872552.0,3674130.0,10574011.0,14776406.0,5649363.0,7528342.0,465466.0


### Constructing arbitrary incidence matrices

Creating an incidence matrix for a given text, with probability values

In [10]:
def create_template_matrix(text):
    index = sorted(list(set(text)))
    columns = index
    num_unique_letters = len(columns)
    data = np.zeros((num_unique_letters, num_unique_letters))
    template_matrix = pd.DataFrame(data, index=index, columns=columns)
    return template_matrix

# testing
#text = 'MATHEMATICS'
#print(create_template_matrix(text))

In [11]:
def get_bigrams(text):
    return re.findall(r'(?=([a-zA-Z]{2}))', text)

def get_bigram_frequencies(text):
    bigrams = get_bigrams(text)
    counts = Counter(bigrams)
    df_counts = pd.DataFrame.from_dict(counts, orient='index').reset_index()
    df_counts = df_counts.rename(columns={'index':'bigram', 0:'frequency'})
    df_counts['first'] = df_counts.bigram.str[0]
    df_counts['second'] = df_counts.bigram.str[1]
    df_counts = df_counts.sort_values('bigram') 
    return df_counts

def get_incidence_matrix(text):
    incidence_matrix = create_template_matrix(text)
    df_bigram_frequencies = get_bigram_frequencies(text)
    for index, row in df_bigram_frequencies.iterrows():
        first, second = row['first'], row['second']
        incidence_matrix.at[first, second] = row['frequency']
    incidence_matrix_probs = incidence_matrix.apply(lambda x: x / incidence_matrix.values.sum())
    incidence_matrix_probs_arr = incidence_matrix_probs.values
    return incidence_matrix_probs_arr

# testing
#text = 'MATHEMATICS'
#print(get_bigrams(text))
#print(get_bigram_frequencies(text))
#print(get_incidence_matrix(text))

Retrieving part of English incidency matrix, with frequency values

In [12]:
def get_subpart_english_incidence_matrix(text, typical_incidence_matrix): # typical_incidence_matrix = entire English bigram matrix
    subpart_matrix = create_template_matrix(text)
    for index, row in subpart_matrix.iterrows():
        for col in subpart_matrix.columns:
            subpart_matrix.at[index, col] = typical_incidence_matrix.at[index, col]
    subpart_matrix_probs = subpart_matrix.apply(lambda x: x / subpart_matrix.values.sum())
    subpart_matrix_probs_arr = subpart_matrix_probs.values
    return subpart_matrix_probs
    
# testing
#text = 'MATHEMATICS'
#print(get_subpart_english_incidence_matrix(text, typical_incidence_matrix_freq_clean))

### Measuring typical-ness of text
Getting Euclidean distance between text's incidence matrix and its associated subpart typical English incidence matrix

In [13]:
def get_dist_from_typical(text, english_bigram_matrix): # english_bigram_matrix is the entire English bigram matrix 
    text_incidence_matrix = get_incidence_matrix(text)
    #print(text_incidence_matrix)
    typical_subpart_incidence_matrix = get_subpart_english_incidence_matrix(text, english_bigram_matrix)
    #print(typical_subpart_incidence_matrix)
    dist = np.linalg.norm(text_incidence_matrix - typical_subpart_incidence_matrix)
    return dist

# testing
#text = 'CAB'
#english_bigram_matrix = typical_incidence_matrix_freq_clean
#print(get_dist_from_typical(text, english_bigram_matrix))

### Constructing encoding and decoding schemes
• Substitution ciphers are random permutations of English letters and spaces  
• Decoding involves repeatedly randomly switching two letters of encoded string, with the goal of minimizing distance from typical incidence matrix

In [14]:
def encode(text):
    return ''.join(random.sample(text, len(text))) # random permutation of text

def swap(s, i, j):
    lst = list(s)
    lst[i], lst[j] = lst[j], lst[i]
    return ''.join(lst)

def exchange(text):
    (i, j) = tuple(random.sample(set(range(len(text))), 2))
    print((i,j))
    return swap(text, i, j) # exchanges two random characters in string

# testing
#text = 'THANKFUL'
#encoded_text = encode(text)
#exchanged_encoded_text = exchange(encoded_text)
#print(encoded_text)
#print(exchanged_encoded_text)

In [58]:
def decode(english_bigram_matrix, text, encoded_text, switch_prob): 
    cur_text = encoded_text
    cur_dist_from_typical = math.inf
    num_exchanges = 0
    
    while cur_text != text: # possible for loop to run many times for long texts
        exchanged_encoded_text = exchange(cur_text)
        print('current text:', cur_text)
        print('exchanged encoded text:', exchanged_encoded_text)
        exchanged_dist_from_typical = get_dist_from_typical(exchanged_encoded_text, english_bigram_matrix) # only text's letters involved
        if exchanged_dist_from_typical <= cur_dist_from_typical:
            cur_dist_from_typical = exchanged_dist_from_typical
            cur_text = exchanged_encoded_text
            num_exchanges += 1
        else:
            drawn_prob = random.random()
            print('drawn probability:', drawn_prob)
            if drawn_prob <= switch_prob: 
                cur_dist_from_typical = exchanged_dist_from_typical 
                cur_text = exchanged_encoded_text 
            else:
                cur_dist_from_typical = get_dist_from_typical(cur_text, english_bigram_matrix)
                print('no exchange')
        
        print('dist from typical (projected):', exchanged_dist_from_typical)
        print('dist from typical (actual):', cur_dist_from_typical)
        print('# exchanges:', num_exchanges)
        
    return num_exchanges # returns the number of informed exchanges needed to resolve to original

In [59]:
# testing
english_bigram_matrix = typical_incidence_matrix_freq_clean
text = 'BATS'
encoded_text = encode(text)
switch_prob = 0.5 # observe over multiple values

print('text:', text)
print('encoded text:', encoded_text)
print(decode(english_bigram_matrix, text, encoded_text, switch_prob))

text: BATS
encoded text: STBA
(1, 0)
current text: STBA
exchanged encoded text: TSBA
dist from typical (projected): 0.610887877658526
dist from typical (actual): 0.610887877658526
# exchanges: 1
(2, 0)
current text: TSBA
exchanged encoded text: BSTA
dist from typical (projected): 0.5101625217955926
dist from typical (actual): 0.5101625217955926
# exchanges: 2
(2, 0)
current text: BSTA
exchanged encoded text: TSBA
drawn probability: 0.8740894867654989
no exchange
dist from typical (projected): 0.610887877658526
dist from typical (actual): 0.5101625217955926
# exchanges: 2
(0, 3)
current text: BSTA
exchanged encoded text: ASTB
dist from typical (projected): 0.4768801252131568
dist from typical (actual): 0.4768801252131568
# exchanges: 3
(1, 2)
current text: ASTB
exchanged encoded text: ATSB
drawn probability: 0.4139909004628163
dist from typical (projected): 0.5300246916287684
dist from typical (actual): 0.5300246916287684
# exchanges: 3
(0, 1)
current text: ATSB
exchanged encoded text: 